In [ ]:
# the first three cells must be executed

import sys
sys.path.insert(0, "..")

In [ ]:
%matplotlib notebook

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import os
from flexrilog import FlexRiGraph

from matplotlib.backend_bases import MouseButton
from matplotlib.collections import LineCollection

class GraphDrawer:
    epsilon = 15  # max pixel distance to count as a vertex hit

    def __init__(self):
        
        _, self.ax = plt.subplots()
        plt.gcf().set_size_inches(9,6)
        self.coll = LineCollection([])
        self.ax.add_collection(self.coll)
        
        self.coll.set_animated(True)
        self.ax.set_xlim(-3, 4)
        self.ax.set_ylim(-3, 4)

        self.ax.set_title('keys: d - DRAWING, m - moving vertices\n  e - delete last edge, v - delete last vertex')
        canvas = self.ax.figure.canvas
        
        self.edges = []
        self.verts = np.array([[0,0]])
        self.labels = [self.ax.text(0,0,0,bbox={'facecolor': 'blue', 'alpha': 0.5, 'pad': 1})]
        
        self.first = None
        self._ind = None  # the active vertex
        self.mode = 'draw'

        canvas.mpl_connect('draw_event', self.on_draw)
        canvas.mpl_connect('button_press_event', self.on_button_press)
        canvas.mpl_connect('key_press_event', self.on_key_press)
        canvas.mpl_connect('button_release_event', self.on_button_release)
        canvas.mpl_connect('motion_notify_event', self.on_mouse_move)
        self.canvas = canvas
        self.background = self.canvas.copy_from_bbox(self.ax.bbox)
        self.update_segments()

    def _print(self,s):
        os.write(1,str(s).encode())
        os.write(1,'\n'.encode())
        
    def report(self):
        self._print('Vertices:')
        self._print(self.verts)
        self._print('Edges:')
        self._print(self.edges)
        self._print('Selected:')
        self._print(self._ind)
        self._print('First')
        self._print(self.first)
        
    def add_vertex(self,x,y):
        self.verts = np.append(self.verts,
                               np.array([[x,y]]), axis=0)
        self.labels.append(self.ax.text(x,y,str(len(self.verts)-1),
                                  bbox={'facecolor': 'blue', 'alpha': 0.5, 'pad': 1}))
        return len(self.verts)-1
    
    def remove_last_vertex(self):
        if len(self.verts)>0:
            self.verts = self.verts[:-1]
            self.labels[-1].remove()
            self.labels = self.labels[:-1]
            w = len(self.verts)
            self.edges = [e for e in self.edges if not w in e]
            
    
    def remove_last_edge(self):
        self.edges = self.edges[:-1]
    
    def add_edge(self,u,v):
        self.edges.append([u,v])
        
    def update_segments(self):
        segments = [[self.verts[u],self.verts[v]] for u,v in self.edges]
        self.coll.set_segments(segments)
        for i,z in enumerate(self.verts):
            self.labels[i].set_position(z)
            self.labels[i].set_bbox({'facecolor': 'blue' if i!=self.first else 'red', 'alpha': 0.5, 'pad': 1})
        
        self.canvas.restore_region(self.background)
        self.ax.draw_artist(self.coll)
        self.canvas.blit(self.ax.bbox)
#         self.report()
        
    def get_ind_under_point(self, event):
        """
        Return the index of the point closest to the event position or *None*
        if no point is within ``self.epsilon`` to the event position.
        """
        xyt = self.ax.transData.transform(self.verts)
        d = np.sqrt((xyt[:,0] - event.x)**2 + (xyt[:,1] - event.y)**2)
        if len(d)==0:
            return None
        ind = d.argmin()
        return ind if d[ind] < self.epsilon else None

    def on_draw(self, event):
        """Callback for draws."""
        self.background = self.canvas.copy_from_bbox(self.ax.bbox)
        self.ax.draw_artist(self.coll)
        self.canvas.blit(self.ax.bbox)

    def on_button_press(self, event):
        """Callback for mouse button presses."""
        if (event.inaxes is None
            or event.button != MouseButton.LEFT
            or not self.ax.get_navigate_mode() is None
           ):
            return
        self._ind = self.get_ind_under_point(event)
        if self.mode == 'draw':
            if self.first == None:
                if self._ind == None:
                    self.first = self.add_vertex(event.xdata, event.ydata)
                    self.update_segments()
                else:
                    self.first = self._ind
                    self.update_segments()
                    self._ind = None

    def on_button_release(self, event):
        """Callback for mouse button releases."""
        if (event.button != MouseButton.LEFT
            or not self.ax.get_navigate_mode() is None):
            return
        if self.mode == 'draw':
            self._ind = self.get_ind_under_point(event)
            if self._ind == None:
                w = self.add_vertex(event.xdata, event.ydata)
            else:
                w = self._ind
            if self.first != w:
                self.add_edge(self.first,w)
            self.first = None
            self._ind = None
            self.update_segments()

    
    def on_key_press(self, event):
        """Callback for key presses."""
        if not event.inaxes:
            return
        if event.key == 'd' or event.key == 'D':
            self.mode = 'draw'
            self.ax.set_title('keys: d - DRAWING, m - moving vertices\n  e - delete last edge, v - delete last vertex')
        if event.key == 'm' or event.key == 'M':
            self.mode = 'move'
            self.ax.set_title('keys: d - drawing, m - MOVING vertices\n  e - delete last edge, v - delete last vertex')
        
        if event.key == 'e' or event.key == 'E':
            self.remove_last_edge()
            self.update_segments()
        if event.key == 'v' or event.key == 'V':
            self.remove_last_vertex()
            self.update_segments()
        self.canvas.draw()

    def on_mouse_move(self, event):
        """Callback for mouse movements."""
        if (self._ind is None
                or event.inaxes is None
                or event.button != MouseButton.LEFT
                or self.mode!='move'
                or not self.ax.get_navigate_mode() is None):
            return
        
        self.verts[self._ind] = np.array([event.xdata, event.ydata])
        self.update_segments()

    def get_graph(self):
        return FlexRiGraph(self.edges,
                           pos={i:p for i,p in enumerate(self.verts)}
                          )


# FlexRiLoG - constructing flexible realizations via edge colorings

Jan Legerský

 *Department of Applied Mathematics, Faculty of Information Technology, Czech Technical University in Prague*
 
Special Semester on Rigidity and Flexibility, workshop Code of Rigidity
 
Jupyter notebook: https://jan.legersky.cz/CodeOfRigidity

### The Package

<span style="font-variant:small-caps;">FlexRiLoG</span>:
Package for <span style="font-variant:small-caps;">SageMath</span>

Available at https://github.com/Legersky/flexrilog.

### Flexible Frameworks

A *realization* (or *placement*) of a graph $G=(V_G, E_G)$ is a map
$$p:V_G \rightarrow \mathbb{R}^2$$
such that $p(u)\neq p(v)$ if $uv \in E_G$.

A *framework* $(G,p)$ is *flexible* if there are infinitely many non-congruent realizations $r$ such that
$$
    ||r(u)-r(v)|| = ||p(u)-p(v)||
$$
for all $uv \in E_G$.

### Flexible $K_{3,3}$

In [ ]:
from flexrilog import FlexRiGraph, GraphMotion
t = var('t')
K33 = FlexRiGraph(graphs.CompleteBipartiteGraph(3,3))
parametrization = {0: vector([-sqrt(2+sin(t)^2),0]),    1: vector([sin(t),0]),    2: vector([sqrt(1+sin(t)^2),0]),
    3: vector([0,sqrt(1+cos(t)*cos(t))]),    4: vector([0,-sqrt(2+cos(t)^2)]),    5: vector([0,cos(t)]),}
motion_K33 = GraphMotion.ParametricMotion(K33,parametrization,'symbolic',check=True)

In [ ]:
show(parametrization[0])

In [ ]:
motion_K33.animation_SVG()

In [ ]:
print(motion_K33.edge_lengths())

### NAC-colorings

A coloring of edges $\delta : E_G \rightarrow \{blue, red\}$ is called a *NAC-coloring*,
if it is surjective and for every cycle in G , either all edges in the
cycle have the same color, or there are at least two blue and two
red edges in the cycle.

In [ ]:
from flexrilog import FlexRiGraph, GraphMotion, GraphGenerator
C4 = FlexRiGraph(graphs.CycleGraph(4))
C4.show_all_NAC_colorings()

### Theorem (Grasegger, L., Schicho, 2019)
A connected graph with at least one edge has a flexible realization if and only if it has a NAC-coloring.

In [ ]:
drawer = GraphDrawer()

In [ ]:
G = drawer.get_graph(); G

In [ ]:
print(G.NAC_colorings())
delta = G.NAC_colorings()[0]

In [ ]:
delta
GraphMotion.GridConstruction(G,delta,zigzag=False).animation_SVG(edge_partition=delta)

In [ ]:
G.has_injective_grid_construction()

### Rotationally symetric TP-frameworks

In [ ]:
from flexrilog import Pframework, CnSymmetricFlexRiGraphCartesianNACs, CnSymmetricFlexRiGraph

In [ ]:
P = GraphGenerator.PenroseFramework(6,numeric=True,radius=10)
plot_args = {'vertex_labels':False,'vertex_size':0, 'edge_thickness':1}
Pplot = P.plot(**plot_args)
filling = point2d([])
for a,b,c,d in P.four_cycles():
    if abs(RR((vector(P._pos[a])-vector(P._pos[b]))*(vector(P._pos[c])-vector(P._pos[b])))) < 0.4:
        filling += polygon([P._pos[v] for v in [a,b,c,d]], color='lightblue', axes=False)
Pplot + filling

In [ ]:
def findPentaStars(P):
    res = []
    for v in P.vertices(sort=False):
        if P.degree(v)==5:
            if sum([1 for u in P.neighbors(v) if P.degree(u)==3])==5:
                res.append([v, [u for u in P.neighbors(v)], P.distance('0',v)])
    return res
braces = [[u,v]  for S in findPentaStars(P) for u,v in Subsets(S[1],2) 
          if ((vector(P._pos[u])-vector(P._pos[v])).norm() < 1.5 and S[2] in [6, 9])]
Pbraced = FlexRiGraph(P.edges(sort=False)+braces,pos=P._pos,check=False)
Pbraced.plot(**plot_args)

In [ ]:
sym = CnSymmetricFlexRiGraphCartesianNACs.Cn_symmetries_gens(Pbraced,5)
PenroseBraced = CnSymmetricFlexRiGraphCartesianNACs(Pbraced, sym)

In [ ]:
PenroseBraced.Cn_symmetric_NAC_colorings()

A NAC-coloring is *Cartesian* if no two vertices are connected by a red and blue path simultaneously.

In [ ]:
deltaP = PenroseBraced.Cn_symmetric_NAC_colorings()[-1]
deltaP.plot(**plot_args)

In [ ]:
M = Pframework(PenroseBraced,PenroseBraced._pos,check=False).flex_from_cartesian_NAC(deltaP)
M.animation_SVG(edge_partition=deltaP,vertex_labels=False,totalTime=24,fileName='penrose')

### Theorem (Grasegger, L., 2024)
A rotationally symmetric TP-framework is flexible if and only if the graph has a rotationally symmetric Cartesian NAC-coloring.

### Reflection symmetry

In [ ]:
from flexrilog import CsSymmetricFlexRiGraph
Gcs = FlexRiGraph([(0, 1), (0, 2), (0, 3), (0, 9), (1, 2), (1, 7), (1, 8), (2, 4), (2, 6),
                   (3, 4), (3, 8), (4, 5), (4, 8), (5, 6), (6, 7), (6, 9), (7, 9)])
Cs_sym = CsSymmetricFlexRiGraph.Cs_symmetries_gens_according_isomorphic_orbits(Gcs)[0]
G_Cs = CsSymmetricFlexRiGraph(Gcs,Cs_sym)
G_Cs.set_symmetric_positions({ 1: [1.03, 0.16], 2: [0, 1.37], 5: [0, 3.49], 
                              6: [1.38, 2.5 ], 7: [1.91, 0.87], 9: [1.17, 1.54]})
G_Cs

### Theorem (Dewar, Grasegger, L., 2024+)
If a reflection-symmetric framework with symmetry $\sigma$ is flexible, then the graph has a *pseudo-RS-coloring*, which is an edge colouring
$\delta:E_G\rightarrow \{red,blue,gold\}$ such that:
 * $ \{ red,blue \}\subseteq \delta(E_G) \subseteq \{ red,blue, gold \}$,
 * changing gold to blue results in a NAC-colouring,
 * changing gold to red results in a NAC-colouring,
 * $\delta(e) = red$ if and only if $\delta(\sigma e) = blue$ for all $e\in E_G$,  and
 * if $\delta(e) = gold$ then $\delta(\sigma e) = gold$ for all $e\in E_G$.

In [ ]:
G_Cs.show_all_pseudoRScolorings()

In [ ]:
from flexrilog import colB, colG, colR
d_Cs = G_Cs.pseudoRScolorings()[1]
GraphMotion.CsSymmetricGridConstruction(G_Cs, d_Cs).animation_SVG(colors=[colR,colB,colG],
                edge_partition=[d_Cs.red_edges(),d_Cs.blue_edges(),d_Cs.golden_edges()])